#### 导入环境

In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [2]:
# 将json文件转为csv文件
df = pd.read_json("./huanhuan.json")
ds = Dataset.from_pandas(df)

In [3]:
df[:3]

,instruction,input,output
0,小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——,,嘘——都说许愿说破是不灵的。
1,这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱...,,你们俩话太多了，我该和温太医要一剂药，好好治治你们。
2,嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。,,出来走走，也是散心。


#### 处理数据集

In [7]:
tokenizer = AutoTokenizer.from_pretrained("./deepseek-ai/deepseek-llm-7b-chat", 
                                          use_fast=False, 
                                          trust_remote_code=True)
tokenizer.padding_size = 'right'
tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LlamaTokenizerFast(name_or_path='./deepseek-ai/deepseek-llm-7b-chat', vocab_size=100000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	100000: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	100001: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	100002: AddedToken("ø", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	100003: AddedToken("ö", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	100004: AddedToken("ú", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	100005: AddedToken("ÿ", rstrip=False, lstrip=False, single_word=False, normali

In [8]:
def process_func(example):
    MAX_LENGTH = 384  # Llama分词器会将一个中文字切分为多个token， 因此需要开放一些最大长度， 保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    # add_special_tokens 不在开头加 special_tokens
    instruction = tokenizer(f"User: {example['instruction']+example['input']}\n\n", 
                            add_special_tokens=False) 

    response = tokenizer(f"Assistant: {example['output']}<｜end▁of▁sentence｜>", 
                         add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    # 因为eos token咱们也是要关注的所以 补充为1
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    if len(input_ids) > MAX_LENGTH:  # 做截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids, 
        "attention_mask": attention_mask, 
        "labels": labels
    }   

In [10]:
tokenize_id = ds.map(process_func, remove_columns=ds.column_names)
tokenize_id

Map:   0%|          | 0/3729 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3729
})

In [11]:
tokenizer.decode(tokenize_id[0]['input_ids'])

'User: 小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——\n\nAssistant: 嘘——都说许愿说破是不灵的。<｜end▁of▁sentence｜><｜end▁of▁sentence｜>'

In [13]:
tokenizer.decode(list(filter(lambda x: x != -100, 
                             tokenize_id[1]["labels"])))

'Assistant: 你们俩话太多了，我该和温太医要一剂药，好好治治你们。<｜end▁of▁sentence｜><｜end▁of▁sentence｜>'

#### 创建模型

In [15]:
import torch
model = AutoModelForCausalLM.from_pretrained('./deepseek-ai/deepseek-llm-7b-chat/', 
                                             trust_remote_code=True, 
                                             torch_dtype=torch.half, 
                                             device_map="auto")
model.generation_config = GenerationConfig.from_pretrained("./deepseek-ai/deepseek-llm-7b-chat")
model.generation_config.pad_token_id = model.generation_config.eos_token_id
model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(102400, 4096)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [16]:
# 开启梯度检查点时， 要执行该方法
model.enable_input_require_grads()

In [17]:
model.dtype

torch.float16

#### Lora

In [18]:
from peft import LoraConfig, TaskType, get_peft_model
config = LoraConfig(
    task_type = TaskType.CAUSAL_LM, 
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], 
    inference_mode = False,  # 训练模式
    r = 8,   # Lora秩
    lora_alpha = 32,  # Lora alpha
    lora_dropout = 0.1
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'q_proj', 'gate_proj', 'o_proj', 'down_proj', 'k_proj', 'v_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False)

In [21]:
model = get_peft_model(model, config)
config


LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='./deepseek-ai/deepseek-llm-7b-chat/', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'q_proj', 'gate_proj', 'o_proj', 'down_proj', 'k_proj', 'v_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False)

In [22]:
model.print_trainable_parameters()

trainable params: 18,739,200 || all params: 6,929,104,896 || trainable%: 0.2704418576606868


#### 配置训练参数

In [25]:
args = TrainingArguments(
    output_dir = "./output/DeepSeek", 
    per_device_train_batch_size = 8, 
    gradient_accumulation_steps = 2, 
    logging_steps = 10, 
    num_train_epochs=3, 
    save_steps = 100, 
    learning_rate=1e-4, 
    save_on_each_node=True, 
    gradient_checkpointing=True
)

In [26]:
trainer = Trainer(
    model=model, 
    args = args, 
    train_dataset = tokenize_id, 
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                            padding=True)
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [27]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/root/miniconda3/envs/mdscpe/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,15.093200
20,0.000000
30,0.000000
40,0.000000
50,0.000000
60,0.000000
70,0.000000
80,0.000000
90,0.000000
100,0.000000


/root/miniconda3/envs/mdscpe/lib/python3.8/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./deepseek-ai/deepseek-llm-7b-chat/ - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/envs/mdscpe/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/envs/mdscpe/lib/python3.8/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./deepseek-ai/deepseek-llm-7b-chat/ - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/envs/mdscpe/lib/python3.8/site-

TrainOutput(global_step=699, training_loss=0.2159259015739561, metrics={'train_runtime': 797.0306, 'train_samples_per_second': 14.036, 'train_steps_per_second': 0.877, 'total_flos': 4.342399452502426e+16, 'train_loss': 0.2159259015739561, 'epoch': 2.99})

In [33]:
# text = "小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——"
# inputs = tokenizer(f"User:{text}\n\n", return_tensors="pt")
# outputs = model.generate(**inputs.to(model.device), max_new_tokens=100)
# result = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(result)
text = "小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的"
inputs = tokenizer(f"User: {text}\n\n", return_tensors="pt")
outputs = model.generate(**inputs.to(model.device), max_new_tokens=100)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [ ]:
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia